## **size**=**`perc_valid_cf_all`**

**Definição do artigo:**
- size = |C|/k
- Onde |C| = número de counterfactuals válidos gerados
- k = número de counterfactuals solicitados

**No código:**



In [ ]:
def perc_valid_cf(cf_list, b, y_val, k=None, y_desidered=None):
    n_val = nbr_valid_cf(cf_list, b, y_val, y_desidered)  # |C| - CFs válidos
    k = len(cf_list) if k is None else k
    res = n_val / k  # |C|/k
    return res



E na função `evaluate_cf_list` do Guidotti ela vem como:


In [ ]:
perc_valid_cf_all_ = perc_valid_cf(cf_list, bb, y_val, k=max_nbr_cf)

# Onde `max_nbr_cf` é o k (número de counterfactuals solicitados)
# e a função nbr_valid_cf é definida como:
def nbr_valid_cf(cf_list, b, y_val, y_desidered=None):
    y_cf = b.predict(cf_list)
    idx = y_cf != y_val if y_desidered is None else y_cf == y_desidered
    val = np.sum(idx)
    return val


**Detalhamento do código:** 

### `nbr_valid_cf()`
1. Faz a predição de todas as instâncias counterfactual (`cf_list`) usando o modelo black box `b`
2. Cria um array booleano `idx` que indica quais counterfactuals são válidos
3. Se `y_desidered` não é especificado, considera válido qualquer CF com classe diferente da original (`y_cf != y_val`)
4. Se `y_desidered` é especificado, considera válido apenas CFs que chegam exatamente naquela classe desejada
5. Retorna a contagem total de CFs válidos

**Interpretação:**
- Um counterfactual é **válido** se, ao ser classificado pelo modelo, produz uma classe diferente da instância original.
- **Quanto maior, melhor**
- Indica quantos dos CFs gerados realmente funcionam como counterfactuals
- Um método perfeito teria `nbr_valid_cf = k` (número de CFs solicitados)
- `perc_valid_cf` normaliza isso em porcentagem: valores próximos a 100% são ideais

### `perc_valid_cf_all()`

**Diferença crucial:**
- `perc_valid_cf`: divide pelo número de CFs **efetivamente gerados**
- `perc_valid_cf_all`: divide por `k` (número **solicitado** de CFs)

---

**Resumo:**
- `nbr_valid_cf` = |C| (número absoluto de CFs válidos)
- `perc_valid_cf` = |C| / |cf_list| (taxa sobre os CFs efetivamente gerados)
- **`perc_valid_cf_all` = |C| / k (size do artigo - taxa sobre k solicitados)**

Portanto, **`perc_valid_cf_all`** é a métrica "size" mencionada no artigo, representando a proporção de counterfactuals válidos em relação ao número k solicitado. Ela ajuda pois alguns métodos podem gerar menos CFs do que o solicitado.


## **Actionability**

corresponde a **`perc_actionable_cf_all`** no codigo

---

### **Definição do Artigo:**
- $act = |{c ∈ C | a_A(c, x)}| / k$
- Onde:
  - $|{c ∈ C | a_A(c, x)}|$ = número de counterfactuals que **podem ser realizados** (respeitam constraints)
  - $a_A(c, x)$ = função que verifica se o counterfactual c é acionável a partir de x
  - $k$ = número de counterfactuals solicitados

---

### **Como funciona no código:**

#### **1. Função base: `nbr_actionable_cf`**
Conta quantos CFs respeitam os constraints (features imutáveis):



In [ ]:
def nbr_actionable_cf(x, cf_list, variable_features):
    nbr_actionable = 0
    nbr_features = cf_list.shape[1]
    for i, cf in enumerate(cf_list):
        constraint_violated = False
        for j in range(nbr_features):
            # Verifica se uma feature foi alterada E não está na lista de features variáveis
            if cf[j] != x[j] and j not in variable_features:
                constraint_violated = True
                break
        if not constraint_violated:
            nbr_actionable += 1
    return nbr_actionable



**Lógica:**
- Para cada counterfactual `cf`:
  - Verifica todas as features modificadas
  - Se alguma feature modificada **NÃO está** em `variable_features` (é imutável), o CF viola constraints
  - Conta apenas CFs que **não violam** nenhum constraint

#### **2. Função de percentual: `perc_actionable_cf`**
Calcula a proporção de CFs acionáveis:



In [ ]:
def perc_actionable_cf(x, cf_list, variable_features, k=None):
    n_val = nbr_actionable_cf(x, cf_list, variable_features)  # |{c ∈ C | aA(c, x)}|
    k = len(cf_list) if k is None else k
    res = n_val / k  # Proporção
    return res



#### **3. Na função `evaluate_cf_list`:**


In [ ]:
perc_actionable_cf_all_ = perc_actionable_cf(x, cf_list, variable_features, k=max_nbr_cf)



Onde `max_nbr_cf` é o **k** solicitado.

### **Resumo:**
- **`nbr_actionable_cf`** = |{c ∈ C | aA(c, x)}| (número absoluto de CFs acionáveis)
- **`perc_actionable_cf`** = proporção sobre CFs efetivamente gerados
- **`perc_actionable_cf_all`** = **act do artigo** = |{c ∈ C | aA(c, x)}| / k

**Exemplo prático:**
- Se k=5, técnica gerou 5 CFs, mas apenas 3 respeitam constraints (não modificaram features imutáveis):
  - `nbr_actionable_cf` = 3
  - `perc_actionable_cf_all` = 3/5 = 0.6 ou 60%
---


## **DETALHAMENTO DAS FUNÇÕES E DISTÂNCIAS IMPLEMENTADAS**

### **1. Funções de Distância Base**

O código implementa múltiplas métricas de distância para acomodar diferentes tipos de dados e necessidades de normalização:

#### **1.1. Distâncias para Features Contínuas**

**Euclidean Distance (L2)**
```python
metric='euclidean'

- Distância padrão no espaço euclidiano: $d(x,y) = \sqrt{\sum_{i=1}^{m}(x_i - y_i)^2}$
- **Sensível à escala**: features com maior magnitude dominam o cálculo
- **Uso**: Quando os dados já estão normalizados ou escala não é problema
- **Implementações**: `distance_l2`, `diversity_l2`, `distance_l2j`, `diversity_l2j`

**MAD (Median Absolute Deviation)**


In [ ]:
metric='mad'

- **Definição**: MAD mede a dispersão robusta de cada feature
  $$MAD_i = \text{median}(|X_i - \text{median}(X_i)|)$$
- **Normalização**: Cada diferença é dividida pelo MAD da respectiva feature
  $$d_{MAD}(x,y) = \sum_{i=1}^{m}\frac{|x_i - y_i|}{MAD_i}$$
- **Vantagens**:
  - Robusto a outliers (usa mediana, não média)
  - Normaliza automaticamente diferentes escalas
  - Não requer normalização prévia dos dados
- **Uso**: **Recomendado** para dados do mundo real com outliers e escalas variadas
- **Implementações**: `distance_mad`, `diversity_mad`, `distance_mh`, `diversity_mh`

**Implementação no código:**


In [ ]:
def mad_cityblock(u, v, mad):
    u = _validate_vector(u)
    v = _validate_vector(v)
    l1_diff = abs(u - v)
    l1_diff_mad = l1_diff / mad  # Normalização por MAD
    return l1_diff_mad.sum()

# Cálculo do MAD
mad = median_absolute_deviation(X[:, continuous_features], axis=0)
mad = np.array([v if v != 0 else 1.0 for v in mad])  # Evita divisão por zero



---

#### **1.2. Distâncias para Features Categóricas**

**Jaccard Distance**


In [ ]:
metric='jaccard'

- **Definição**: Mede dissimilaridade baseada em conjuntos
  $$d_{Jaccard}(x,y) = 1 - \frac{|A \cap B|}{|A \cup B|}$$
- **Características**:
  - Varia entre 0 (idênticos) e 1 (completamente diferentes)
  - Considera a presença/ausência de valores
  - Apropriado para dados binários ou one-hot encoded
- **Uso**: Quando features categóricas são representadas como vetores binários
- **Implementações**: `distance_j`, `diversity_j`, `distance_l2j`, `diversity_l2j`

**Hamming Distance**


In [ ]:
metric='hamming'

- **Definição**: Proporção de features diferentes
  $$d_{Hamming}(x,y) = \frac{1}{m}\sum_{i=1}^{m}\mathbb{1}_{x_i \neq y_i}$$
- **Características**:
  - Varia entre 0 (idênticos) e 1 (todas as features diferentes)
  - Trata cada feature igualmente
  - Mais intuitivo para dados categóricos gerais
- **Uso**: **Recomendado** para features categóricas nominais
- **Implementações**: `distance_h`, `diversity_h`, `distance_mh`, `diversity_mh`

---

### **2. Distâncias Híbridas (Dados Mistos)**

Para datasets com features contínuas **e** categóricas, o código implementa combinações ponderadas:

#### **2.1. L2 + Jaccard (distance_l2j / diversity_l2j)**


In [ ]:
def distance_l2j(x, cf_list, continuous_features, categorical_features, 
                 ratio_cont=None, agg=None):
    dist_cont = continuous_distance(x, cf_list, continuous_features, 
                                   metric='euclidean', X=None, agg=agg)
    dist_cate = categorical_distance(x, cf_list, categorical_features, 
                                    metric='jaccard', agg=agg)
    
    # Ponderação proporcional ao número de features
    if ratio_cont is None:
        ratio_continuous = len(continuous_features) / nbr_features
        ratio_categorical = len(categorical_features) / nbr_features
    
    dist = ratio_continuous * dist_cont + ratio_categorical * dist_cate
    return dist



**Características:**
- Combina Euclidean (contínuas) + Jaccard (categóricas)
- Ponderação automática ou manual via `ratio_cont`
- **Limitação**: Sensível à escala das contínuas

#### **2.2. MAD + Hamming (distance_mh / diversity_mh) - RECOMENDADA**


In [ ]:
def distance_mh(x, cf_list, continuous_features, categorical_features, 
                X, ratio_cont=None, agg=None):
    dist_cont = continuous_distance(x, cf_list, continuous_features, 
                                   metric='mad', X=X, agg=agg)
    dist_cate = categorical_distance(x, cf_list, categorical_features, 
                                    metric='hamming', agg=agg)
    
    dist = ratio_continuous * dist_cont + ratio_categorical * dist_cate
    return dist



**Vantagens sobre L2J:**
- **Robustez**: MAD não é afetado por outliers
- **Normalização automática**: Não requer pré-processamento
- **Interpretabilidade**: Hamming é mais intuitivo que Jaccard
- **Escalabilidade**: Melhor performance em datasets grandes

---

### **3. Funções de Agregação (agg)**

Todas as funções de distância/diversidade suportam agregação:



In [ ]:
agg=None ou agg='mean'  # Padrão: média aritmética
agg='min'               # Mínimo (CF mais próximo/similar)
agg='max'               # Máximo (CF mais distante/diverso)



**Aplicações:**
- **Métricas principais** usam `mean` (implementam fórmulas do artigo)
- **Métricas auxiliares** usam `min`/`max` para análise detalhada:
  - `distance_mh_min`: CF mais próximo (best-case)
  - `distance_mh_max`: CF mais distante (worst-case)
  - `diversity_mh_min`: Par de CFs mais similar
  - `diversity_mh_max`: Par de CFs mais diverso

---

### **4. Comparação das Implementações**

| Métrica | Função Contínuas | Função Categóricas | Normalização | Robustez | Uso Recomendado |
|---------|------------------|-------------------|--------------|----------|-----------------|
| **distance_l2** | Euclidean | - | Manual | Baixa | Dados normalizados |
| **distance_mad** | MAD | - | Automática | Alta | Contínuas com outliers |
| **distance_j** | - | Jaccard | N/A | Média | Categóricas binárias |
| **distance_h** | - | Hamming | N/A | Alta | Categóricas nominais |
| **distance_l2j** | Euclidean | Jaccard | Manual | Baixa | Dados mistos normalizados |
| **distance_mh** | MAD | Hamming | Automática | **Alta** | **Dados mistos gerais** |

### **Resumo Final:**

1. **MAD** é preferível à Euclidean por ser robusta e auto-normalizar
2. **Hamming** é mais intuitiva que Jaccard para categóricas gerais
3. **distance_mh/diversity_mh** são as implementações **mais robustas** para dados mistos
4. Métricas alternativas (l2, l2j) estão disponíveis para **comparação** ou **casos específicos**
5. O parâmetro `agg` permite análises **detalhadas** além da média
6. Sempre use o **conjunto de treino X** para calcular MAD (consistência)

---


## **Implausibility** 

corresponde a função **`plausibility_nbr_cf`**

### **Definição do Artigo:**
- $impl = (\frac{1}{|C|})\sum_{c∈C}{min_{(x∈X)}d(c, x)}$
- Onde:
  - |C| = número de counterfactuals gerados
  - $min_{(x∈X)}d(c, x)$ = distância de cada CF para a instância mais próxima no conjunto de referência X
  - Quanto menor, melhor

### **Como funciona no código:**

#### **1. Função base: `plausibility`**
Calcula a soma das distâncias de cada CF para a instância mais próxima no conjunto de referência:



In [ ]:
def plausibility(x, bb, cf_list, X_test, y_pred, continuous_features_all,
                 categorical_features_all, X_train, ratio_cont):
    sum_dist = 0.0
    for cf in cf_list:
        # 1. Prediz a classe do counterfactual
        y_cf_val = bb.predict(cf.reshape(1, -1))[0]
        
        # 2. Filtra X_test para instâncias da mesma classe que o CF
        X_test_y = X_test[y_cf_val == y_pred]
        
        # 3. Calcula distâncias e encontra o índice da instância mais próxima
        neigh_dist = distance_mh(x.reshape(1, -1), X_test_y, continuous_features_all,
                        categorical_features_all, X_train, ratio_cont)
        idx_neigh = np.argsort(neigh_dist)[0]
        closest = X_test_y[idx_neigh]
        
        # 4. Calcula distância do CF para a instância mais próxima
        d = distance_mh(cf, closest.reshape(1, -1), continuous_features_all,
                        categorical_features_all, X_train, ratio_cont)
        sum_dist += d
    
    return sum_dist  # Σ(c∈C) min(x∈X) d(c, x)



**Observação importante:** A implementação busca a instância mais próxima **da mesma classe predita que o CF**, tornando a métrica mais refinada.

#### **2. Na função `evaluate_cf_list`:**
Várias variações de plausibility são calculadas:



In [ ]:
plausibility_sum = plausibility(...)  # Soma total

# Diferentes normalizações:
plausibility_max_nbr_cf_ = plausibility_sum / max_nbr_cf           # Divide por k solicitado
plausibility_nbr_cf_ = plausibility_sum / nbr_cf_                  # Divide por CFs gerados
plausibility_nbr_valid_cf_ = plausibility_sum / nbr_valid_cf_      # Divide por CFs válidos
plausibility_nbr_actionable_cf_ = plausibility_sum / nbr_actionable_cf_
plausibility_nbr_valid_actionable_cf_ = plausibility_sum / nbr_valid_actionable_cf_



---

### **Resumo:**
- **`plausibility_sum`** = $\sum_{c∈C}{min_{(x∈X)}d(c, x)}$ (soma total)
- **`plausibility_nbr_cf`** = **impl do artigo**
- Variações alternativas:
  - `plausibility_nbr_valid_cf` = normaliza apenas pelos CFs válidos
  - `plausibility_max_nbr_cf` = normaliza por k solicitado

**Exemplo prático:**
- Se gerou 5 CFs e a soma das distâncias mínimas é 10.0:
  - `plausibility_sum` = 10.0
  - `plausibility_nbr_cf` = 10.0 / 5 = 2.0 (implausibility média)

**Interpretação:** Valores baixos indicam que os CFs estão próximos de instâncias reais do conjunto de referência (mais plausíveis). Valores altos indicam CFs distantes da população conhecida (menos plausíveis/mais implausíveis).

---

Seguindo a definição do artigo, as métricas de **Dissimilarity** correspondem a:

## **$dis_{dist}$ (Distance Dissimilarity)**

### **Métricas correspondentes no codigo:**
- **`distance_mh`** (distancia para dados mistos)
- **`distance_l2j`** (alternativa para dados mistos)
- **`distance_mad`** (apenas features contínuas)
- **`distance_l2`** (apenas features contínuas)

### **Definição do Artigo:**
- **$dist_{dist} = (\frac{1}{|C|})\sum_{x`∈C}{d(x,x`)}$**
- Distância média entre x original e cada counterfactual
- Quanto menor, melhor (CFs mais próximos ao original)

### **Como funciona no código:**



In [ ]:
distance_mh_ = distance_mh(x, cf_list, continuous_features_all, 
                          categorical_features_all, X_train, ratio_cont)



A função calcula:
1. Distância de `x` para cada `cf` em `cf_list`
2. Por padrão (`agg='mean'`), retorna a **média** das distâncias
3. Usa MAD para contínuas + Hamming para categóricas

**Variações disponíveis:**
- `distance_mh_min`: distância mínima (CF mais próximo)
- `distance_mh_max`: distância máxima (CF mais distante)

---

## $dis_{count}$ (Count Dissimilarity)

### **Métrica correspondente: `avg_nbr_changes`**

### **Definição do Artigo:**
- **$dist_{count} = (\frac{1}{|C|m})\sum_{c∈C}\sum_{i=1}^{m}{1_{c_i≠x_i}}$**
- o número médio de características alteradas entre um c contrafactual e x
- Quanto menor, melhor (menos mudanças necessárias)

### **Como funciona no código:**

#### **1. Função auxiliar: `nbr_changes_per_cf`**
Conta quantas features foram alteradas em cada CF:



In [ ]:
def nbr_changes_per_cf(x, cf_list, continuous_features):
    nbr_features = cf_list.shape[1]
    nbr_changes = np.zeros(len(cf_list))
    for i, cf in enumerate(cf_list):
        for j in range(nbr_features):
            if cf[j] != x[j]:
                # Conta 1 para contínua, 0.5 para categórica
                nbr_changes[i] += 1 if j in continuous_features else 0.5
    return nbr_changes



#### **2. Função principal: `avg_nbr_changes`**
Implementa exatamente a fórmula do artigo:



In [ ]:
def avg_nbr_changes(x, cf_list, nbr_features, continuous_features):
    val = np.sum(nbr_changes_per_cf(x, cf_list, continuous_features))
    nbr_cf, _ = cf_list.shape
    return val / (nbr_cf * nbr_features)  # Divide por |C| * m



**No `evaluate_cf_list`:**


In [ ]:
avg_nbr_changes_ = avg_nbr_changes(x, cf_list, nbr_features, continuous_features_all)

**Interpretação:** Ambas medem **proximidade** - CFs devem ser diferentes o suficiente para mudar a predição, mas próximos o bastante para serem úteis e interpretáveis.

---

## **Diversity** correspondem a:

## **1. div_dist (Distance-Based Diversity)**

### **Métricas correspondentes:**
- **`diversity_mh`** (dados mistos)
- **`diversity_l2j`** (alternativa para dados mistos)
- **`diversity_mad`** (apenas features contínuas)
- **`diversity_l2`** (apenas features contínuas)

### **Definição do Artigo:**
$$\text{divdist} = \frac{1}{|C|^2} \sum_{c \in C} \sum_{c' \in C} d(c, c')$$

- Distância média entre **todos os pares** de counterfactuals
- Quanto maior, melhor (CFs mais diversos/diferentes entre si)

In [ ]:
def diversity_mh(cf_list, continuous_features, categorical_features, X, ratio_cont=None, agg=None):
    nbr_features = cf_list.shape[1]
    # Diversidade em features contínuas (MAD)
    dist_cont = continuous_diversity(cf_list, continuous_features, metric='mad', X=X, agg=agg)
    # Diversidade em features categóricas (Hamming)
    dist_cate = categorical_diversity(cf_list, categorical_features, metric='hamming', agg=agg)
    
    # Combinação ponderada
    if ratio_cont is None:
        ratio_continuous = len(continuous_features) / nbr_features
        ratio_categorical = len(categorical_features) / nbr_features
    else:
        ratio_continuous = ratio_cont
        ratio_categorical = 1.0 - ratio_cont
    
    dist = ratio_continuous * dist_cont + ratio_categorical * dist_cate
    return dist

**Variações disponíveis:**
- `diversity_mh_min`: menor distância entre pares (CFs mais similares)
- `diversity_mh_max`: maior distância entre pares (CFs mais distantes)

---

## **2. div_count (Count-Based Diversity)**

**Métrica correspondente: `count_diversity_all`**

**Definição do Artigo:**
$$\text{divcount} = \frac{1}{|C|^2 m} \sum_{c \in C} \sum_{c' \in C} \sum_{i=1}^{m} \mathbb{1}_{c_i \neq c'_i}$$

- Proporção média de features diferentes entre pares de CFs
- Normalizado por: número de pares × número de features
- Quanto maior, melhor (CFs modificam diferentes features)

**Como funciona no código:**

**1. Função base: `count_diversity`**
Implementa a lógica de contagem:



In [ ]:
def count_diversity(cf_list, features, nbr_features, continuous_features):
    nbr_cf = cf_list.shape[0]
    nbr_changes = 0
    
    # Loop sobre todos os pares (i, j)
    for i in range(nbr_cf):
        for j in range(i+1, nbr_cf):  # Evita duplicatas
            # Para cada feature
            for k in features:
                if cf_list[i][k] != cf_list[j][k]:
                    # Peso: 1 para contínua, 0.5 para categórica
                    nbr_changes += 1 if k in continuous_features else 0.5
    
    # Normalização: divide por |C|² * m
    return nbr_changes / (nbr_cf * nbr_cf * nbr_features)



**Observação:** O loop usa `range(i+1, nbr_cf)` para contar cada par uma vez, mas a divisão por `nbr_cf * nbr_cf` normaliza considerando todos os pares ordenados, equivalente à fórmula do artigo.

#### **2. Função wrapper: `count_diversity_all`**
Aplica a todas as features:



In [ ]:
def count_diversity_all(cf_list, nbr_features, continuous_features):
    # Aplica count_diversity a TODAS as features
    return count_diversity(cf_list, range(cf_list.shape[1]), nbr_features, continuous_features)



**No `evaluate_cf_list`:**


In [ ]:
count_diversity_all_ = count_diversity_all(cf_list, nbr_features, continuous_features_all)



---

### **Resumo Comparativo:**

| Métrica do Artigo | Implementação | Fórmula | Direção |
|-------------------|---------------|---------|---------|
| **div_dist** | `diversity_mh` | $\frac{1}{\|C\|^2} \sum_{c \in C} \sum_{c' \in C} d(c, c')$ | ↑ maior melhor |
| **div_count** | `count_diversity_all` | $\frac{1}{\|C\|^2 m} \sum_{c \in C} \sum_{c' \in C} \sum_{i=1}^{m} \mathbb{1}_{c_i \neq c'_i}$ | ↑ maior melhor |

**Diferença chave:**
- **div_dist**: mede diversidade no **espaço de features** (distância geométrica)
- **div_count**: mede diversidade na **contagem de mudanças** (combinatória)

**Interpretação:** Alta diversidade significa que o usuário tem múltiplas opções diferentes para reverter a predição negativa, cada uma modificando diferentes combinações de features.

---

Seguindo a definição do artigo, a métrica de **Discriminative Power** corresponde a:

## **Discriminative Power (dipo)**

### **Métricas correspondentes:**
- **`accuracy_knn_sklearn`** (implementação usando sklearn)
- **`accuracy_knn_dist`** (implementação manual com distâncias customizadas)

### **Definição do Artigo:**
**dipo** = acurácia de um classificador 1-Nearest Neighbor treinado com $C \cup \{x\}$ para classificar instâncias em $X_= \cup X_{\neq}$

Onde:
- $X_= \subset X$: k instâncias mais próximas de x com $b(X_=) = b(x)$ (mesma classe)
- $X_{\neq} \subset X$: k instâncias mais próximas de x com $b(X_{\neq}) \neq b(x)$ (classe diferente)
- Quanto maior, melhor (CFs distinguem bem entre classes)

---

### **Como funciona no código:**

#### **1. Função auxiliar: `select_test_knn`**
Seleciona o conjunto de teste $X_= \cup X_{\neq}$:



In [ ]:
def select_test_knn(x, b, X_test, continuous_features, categorical_features, 
                    scaler, test_size=5, get_normalized=False):
    # Predições
    y_val = b.predict(x.reshape(1, -1))
    y_test = b.predict(X_test)
    
    # Normalização
    nx = scaler.transform(x.reshape(1, -1))
    nX_test = scaler.transform(X_test)
    
    # Calcular distâncias para X= (mesma classe)
    dist_f = euclidean_jaccard(nx, nX_test[y_test == y_val], 
                               continuous_features, categorical_features)
    
    # Calcular distâncias para X≠ (classe diferente)
    dist_cf = euclidean_jaccard(nx, nX_test[y_test != y_val], 
                                continuous_features, categorical_features)
    
    # Selecionar k=test_size instâncias mais próximas de cada classe
    index_f = np.argsort(dist_f)[0][:test_size].tolist()   # X=
    index_cf = np.argsort(dist_cf)[0][:test_size].tolist() # X≠
    
    # Combinar: X= ∪ X≠
    index = np.array(index_f + index_cf)
    
    if get_normalized:
        return X_test[index], nX_test[index]
    return X_test[index]



**Resultado:** Retorna 2×k instâncias (k da mesma classe + k da classe oposta)


#### **2. Implementação A: `accuracy_knn_sklearn`**
Usa sklearn para treinar e avaliar o 1NN:



In [ ]:
def accuracy_knn_sklearn(x, cf_list, b, X_test, continuous_features, 
                        categorical_features, scaler, test_size=5):
    # 1. Preparar conjunto de treino: C ∪ {x}
    clf = KNeighborsClassifier(n_neighbors=1)  # 1-Nearest Neighbor
    X_train = np.vstack([x.reshape(1, -1), cf_list])
    y_train = b.predict(X_train)
    
    # 2. Treinar o 1NN
    clf.fit(X_train, y_train)
    
    # 3. Selecionar conjunto de teste: X= ∪ X≠
    X_test_knn = select_test_knn(x, b, X_test, continuous_features, 
                                  categorical_features, scaler, test_size)
    
    # 4. Obter predições reais e do 1NN
    y_test = b.predict(X_test_knn)
    y_pred = clf.predict(X_test_knn)
    
    # 5. Calcular acurácia (discriminative power)
    return accuracy_score(y_test, y_pred)



---

#### **3. Implementação B: `accuracy_knn_dist`**
Implementação manual com cálculo explícito de distâncias:



In [ ]:
def accuracy_knn_dist(x, cf_list, b, X_test, continuous_features, 
                     categorical_features, scaler, test_size=5):
    # 1. Preparar conjunto de treino: C ∪ {x}
    X_train = np.vstack([x.reshape(1, -1), cf_list])
    y_train = b.predict(X_train)
    nX_train = scaler.transform(X_train)
    
    # 2. Selecionar conjunto de teste: X= ∪ X≠
    X_test_knn, nX_test_knn = select_test_knn(x, b, X_test, 
                                              continuous_features, 
                                              categorical_features, 
                                              scaler, test_size, 
                                              get_normalized=True)
    y_test = b.predict(X_test_knn)
    
    # 3. Classificação manual: para cada instância de teste
    y_pred = list()
    for nx_test in nX_test_knn:
        # Calcular distância para todos no conjunto de treino
        dist = euclidean_jaccard(nx_test, nX_train, 
                                continuous_features, categorical_features)
        # Encontrar o vizinho mais próximo (1NN)
        idx = np.argmin(dist)
        # Atribuir classe do vizinho mais próximo
        y_pred.append(y_train[idx])
    
    # 4. Calcular acurácia (discriminative power)
    return accuracy_score(y_test, y_pred)



---

### **Resumo:**

| Métrica do Artigo | Implementação | Descrição | Direção |
|-------------------|---------------|-----------|---------|
| **dipo** | `accuracy_knn_sklearn` | Acurácia 1NN (sklearn) | ↑ maior melhor |
| **dipo** | `accuracy_knn_dist` | Acurácia 1NN (manual) | ↑ maior melhor |

**No `evaluate_cf_list`:**


In [ ]:
accuracy_knn_sklearn_ = accuracy_knn_sklearn(x, cf_list, bb, X_test, 
                                            continuous_features_all,
                                            categorical_features_all, 
                                            scaler, test_size=5)

accuracy_knn_dist_ = accuracy_knn_dist(x, cf_list, bb, X_test, 
                                      continuous_features_all,
                                      categorical_features_all, 
                                      scaler, test_size=5)



**Interpretação:** 
- **Alta acurácia (próxima a 1.0)**: Os CFs formam uma boa fronteira de decisão, conseguindo distinguir bem entre as classes
- **Baixa acurácia (próxima a 0.5)**: Os CFs não definem bem a fronteira, sugerindo que não são discriminativos ou estão confusos

**Por que 1NN?** Pela simplicidade e conexão com o raciocínio humano baseado em exemplos - decisões são tomadas comparando com o caso mais similar conhecido.

---

## **Runtime**

### **Definição:**

**Runtime** mede o **tempo decorrido** necessário para o explainer gerar os counterfactuals. É uma métrica de **eficiência computacional**.

$$\text{runtime} = t_{\text{end}} - t_{\text{start}}$$

Onde:
- $t_{\text{start}}$ = timestamp no início da geração dos CFs
- $t_{\text{end}}$ = timestamp ao final da geração dos CFs
- Medido em **segundos**
- **Quanto menor, melhor**

### **Como funciona no código:**

A métrica `runtime` não é calculada dentro de cf_metrics.ipynb, mas sim no **script de experimentos principal** que chama os métodos de geração de counterfactuals. O padrão típico seria:



In [ ]:
import time

# Antes de gerar CFs
time_start = time.time()

# Geração dos counterfactuals (chamada ao método)
cf_list = explainer.explain(x, k=5)  # Gera k counterfactuals

# Após geração
time_end = time.time()

# Calcula runtime
runtime = time_end - time_start



### **Composição no cf_metrics.ipynb:**

No arquivo cf_metrics.ipynb, há **três métricas de tempo** nas colunas:



In [ ]:
columns = [..., 'time_train', 'time_test', 'runtime', ...]

---

## **TABELA RESUMO - MÉTRICAS DE COUNTERFACTUALS**

### **Métricas do Artigo de Guidotti**

| Métrica | Implementação | Equação | Interpretação | Objetivo |
|---------|---------------|---------|-------------|----------|
| **Size** | `perc_valid_cf_all` | $\frac{\|C\|}{k}$ | Proporção de CFs válidos gerados | **Maximizar** ↑ |
| **Actionability** | `perc_actionable_cf_all` | $\frac{\|\\{c \in C \| a_A(c,x)\\}\|}{k}$ | Proporção de CFs que respeitam constraints | **Maximizar** ↑ |
| **Implausibility** | `plausibility_nbr_cf` | $\frac{1}{\|C\|}\sum_{c\in C}\min_{x\in X}d(c,x)$ | Distância média dos CF para as instâncias mais próximas no conjunto de referência X | **Minimizar** ↓ |
| **Dissimilarity_dist** | `distance_mh` | $\frac{1}{\|C\|}\sum_{c\in C}d(x,c)$ | Distância média entre x e CFs | **Minimizar** ↓ |
| **Dissimilarity_count** | `avg_nbr_changes` | $\frac{1}{\|C\|m}\sum_{c\in C}\sum_{i=1}^{m}\mathbb{1}_{c_i\neq x_i}$ | Proporção de features modificadas | **Minimizar** ↓ |
| **Diversity_dist** | `diversity_mh` | $\frac{1}{\|C\|^2}\sum_{c\in C}\sum_{c'\in C}d(c,c')$ | Distância média entre pares de CFs | **Maximizar** ↑ |
| **Diversity_count** | `count_diversity_all` | $\frac{1}{\|C\|^2 m}\sum_{c\in C}\sum_{c'\in C}\sum_{i=1}^{m}\mathbb{1}_{c_i\neq c'_i}$ | Proporção de features diferentes entre CFs | **Maximizar** ↑ |
| **Discriminative Power (Dipo)** | `accuracy_knn_sklearn` | Acurácia 1NN em $X_= \cup X_{\neq}$ | Capacidade de distinguir entre duas classes diferentes usando apenas os contrafactuais em C | **Maximizar** ↑ |
| **Runtime** | `runtime` | $t_{end} - t_{start}$ | Tempo de execução (segundos) | **Minimizar** ↓ |

---

### **Categorias de Métricas:**

#### **1. Validade e Aplicabilidade** (devem ser altas)
- **Size**: Garantir que CFs válidos sejam gerados
- **Actionability**: Garantir que CFs sejam implementáveis

#### **2. Proximidade** (devem ser baixas)
- **dis_dist**: CFs próximos ao original (mudanças mínimas)
- **dis_count**: Poucas features modificadas (sparsity)
- **Implausibility**: CFs próximos a instâncias reais

#### **3. Diversidade** (deve ser alta)
- **div_dist**: CFs geometricamente diversos
- **div_count**: CFs modificam diferentes features

#### **4. Qualidade da Explicação** (deve ser alta)
- **Discriminative Power**: CFs definem bem a fronteira de decisão

#### **5. Robustez** (deve ser baixa)
- **Instability**: CFs consistentes sob perturbações

#### **6. Eficiência** (deve ser baixa)
- **Runtime**: Tempo computacional aceitável

---

## **Notas Importantes**

1. **Normalização**: Todas as métricas de distância dependem da escala dos dados
   - `distance_mh` usa MAD (robusto a outliers)
   - Valores absolutos variam por dataset

2. **Contexto Experimental**:
   - Hardware: Ubuntu 20.04, 252GB RAM, Intel i9 3.30GHz × 36
   - Runtime deve ser comparado apenas no mesmo hardware

3. **Implementações Alternativas**:
   - `distance_l2j` vs `distance_mh`: L2+Jaccard vs MAD+Hamming
   - `accuracy_knn_sklearn` vs `accuracy_knn_dist`: sklearn vs implementação manual

4. **Métricas Complementares**:
   - Sempre analisar múltiplas métricas simultaneamente
   - Nenhuma métrica isolada captura toda a qualidade dos CFs

---

---

## **VERIFICATION REPORT: Metric Implementations (Updated)**

**Comparison between:**
1. **Theory** (as defined in this notebook / Guidotti's paper)
2. **CounterFactualMetrics.py** (your repo implementation)
3. **ECE/cf_eval/metrics.py** (original source from Riccotti)

---

### **1. Size (`perc_valid_cf_all`) ✅ CORRECT**

| Aspect | Theory | CounterFactualMetrics.py | ECE/cf_eval/metrics.py |
|--------|--------|---------------------------|-------------------------|
| **Formula** | $\frac{\|C\|}{k}$ | `n_val / k` | `n_val / k` |
| **Implementation** | Count CFs where prediction ≠ original (or = desired) | Same | Same |

**Verdict:** ✅ **Both implementations match the theory exactly.**

---

### **2. Actionability (`perc_actionable_cf_all`) ✅ CORRECT**

| Aspect | Theory | CounterFactualMetrics.py | ECE/cf_eval/metrics.py |
|--------|--------|---------------------------|-------------------------|
| **Formula** | $\frac{\|\\{c \in C \| a_A(c,x)\\}\|}{k}$ | `n_val / k` | `n_val / k` |
| **Logic** | CF is actionable if it only modifies features in `variable_features` | Same | Same |

**Verdict:** ✅ **Both implementations match the theory exactly.**

---

### **3. Implausibility (`plausibility_nbr_cf`) ✅ FIXED**

| Aspect | Theory | CounterFactualMetrics.py | ECE/cf_eval/metrics.py |
|--------|--------|---------------------------|-------------------------|
| **Formula** | $\frac{1}{\|C\|}\sum_{c\in C}\min_{x'\in X}d(c,x')$ | ✅ **FIXED** - Now computes distance from CF to nearest neighbor | 🔴 Bug remains |

**Previous Bug:** The code found the neighbor closest to **x** (original sample), not closest to **cf** (the counterfactual).

**Fix Applied:** Changed `distance_mh(x.reshape(1, -1), X_test_y, ...)` to `distance_mh(cf.reshape(1, -1), X_test_y, ...)` and simplified by using `np.min()` directly.

---

### **4. Dissimilarity_dist (`distance_mh`) ✅ CORRECT**

| Aspect | Theory | CounterFactualMetrics.py | ECE/cf_eval/metrics.py |
|--------|--------|---------------------------|-------------------------|
| **Formula** | $\frac{1}{\|C\|}\sum_{c\in C}d(x,c)$ | Mean of `cdist(x, cf_list)` | Same |
| **Distance** | MAD + Hamming (weighted) | `ratio_cont * mad_dist + ratio_cat * hamming_dist` | Same |

**Verdict:** ✅ **Both implementations match the theory exactly.**

---

### **5. Dissimilarity_count (`avg_nbr_changes`) ✅ CORRECT**

| Aspect | Theory | CounterFactualMetrics.py | ECE/cf_eval/metrics.py |
|--------|--------|---------------------------|-------------------------|
| **Formula** | $\frac{1}{\|C\|m}\sum_{c\in C}\sum_{i=1}^{m}\mathbb{1}_{c_i\neq x_i}$ | `sum(changes) / (nbr_cf * nbr_features)` | Same |

**Note:** Both implementations weight continuous features as 1.0 and categorical as 0.5. This weighting is a design choice not explicitly in Guidotti's formula, but is consistent between both implementations.

**Verdict:** ✅ **Implementations are consistent with each other and reasonable.**

---

### **6. Diversity_dist (`diversity_mh`) ✅ CORRECT**

| Aspect | Theory | CounterFactualMetrics.py | ECE/cf_eval/metrics.py |
|--------|--------|---------------------------|-------------------------|
| **Formula** | $\frac{1}{\|C\|^2}\sum_{c\in C}\sum_{c'\in C}d(c,c')$ | Mean of `pdist(cf_list)` | Same |

**Note:** Both use `pdist` which returns pairwise distances for unique pairs. The `mean` aggregation averages over $\frac{\|C\|(\|C\|-1)}{2}$ pairs. Since it's symmetric, this is mathematically correct.

**Verdict:** ✅ **Both implementations match the theory correctly.**

---

### **7. Diversity_count (`count_diversity_all`) ✅ CORRECT (Your version)**

| Aspect | Theory | CounterFactualMetrics.py | ECE/cf_eval/metrics.py |
|--------|--------|---------------------------|-------------------------|
| **Formula** | $\frac{1}{\|C\|^2 m}\sum_{c\in C}\sum_{c'\in C}\sum_{i=1}^{m}\mathbb{1}_{c_i\neq c'_i}$ | ✅ `k in continuous_features` | 🔴 Bug: `j in continuous_features` |

**ECE Bug:** Line 266 uses `j in continuous_features` where `j` is the CF index, not the feature index `k`.

**Your version is correct** - it properly uses `k in continuous_features`.

---

### **8. Discriminative Power (`accuracy_knn_sklearn`) ✅ FIXED**

| Aspect | Theory | CounterFactualMetrics.py | ECE/cf_eval/metrics.py |
|--------|--------|---------------------------|-------------------------|
| **Method** | 1-NN trained on $C \cup \{x\}$, tested on $X_= \cup X_{\neq}$ | ✅ **FIXED** | 🔴 Bug remains |

**Previous Bug in `select_test_knn`:** Index mapping error - indices from filtered arrays were used directly on `X_test` instead of mapping back to original indices.

**Example of bug:**
```python
# If y_test = [0, 1, 0, 1, 0] and y_val = 0
# same_class indices are [0, 2, 4] in X_test
# But after filtering, if sorted index is [1], code returned X_test[1] instead of X_test[2]
```

**Fix Applied:** Now properly maps filtered array indices back to original `X_test` indices using `np.where()`.

---

### **9. Runtime** ✅ N/A

Runtime is measured externally in the experiment script, not computed by these metric functions.

---

## **SUMMARY OF FIXES APPLIED**

| Metric | Issue | Status | Impact |
|--------|-------|--------|--------|
| **Implausibility** | Was finding neighbor of x, not cf | ✅ **FIXED** | High - was measuring wrong thing |
| **Discriminative Power** | Index mapping bug in `select_test_knn` | ✅ **FIXED** | High - was using wrong test samples |

---

## **REMAINING ECE BUGS (Not Fixed - External Repo)**

| Metric | Bug |
|--------|-----|
| **Implausibility** | Same bug as ours was (uses x instead of cf) |
| **Diversity_count** | Uses `j` (CF index) instead of `k` (feature index) |
| **Discriminative Power** | Same index mapping bug |

---

## **FINAL STATUS**

| Metric | CounterFactualMetrics.py | ECE/cf_eval/metrics.py |
|--------|--------------------------|------------------------|
| Size | ✅ Correct | ✅ Correct |
| Actionability | ✅ Correct | ✅ Correct |
| Implausibility | ✅ **FIXED** | 🔴 Bug |
| Dissimilarity_dist | ✅ Correct | ✅ Correct |
| Dissimilarity_count | ✅ Correct | ✅ Correct |
| Diversity_dist | ✅ Correct | ✅ Correct |
| Diversity_count | ✅ Correct | 🔴 Bug |
| Discriminative Power | ✅ **FIXED** | 🔴 Bug |
| Runtime | ✅ N/A | ✅ N/A |

**All 9 metrics in your CounterFactualMetrics.py are now verified correct! ✅**